# **imports and variables:**

In [52]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

!pip install plotly
import plotly.graph_objects as go

In [53]:
sp100 = [
    "AAPL", "ABBV", "ABT", "ACN", "ADBE", "AIG", "AMD", "AMGN", "AMT", "AMZN",
    "ANET", "APA", "APD", "AVGO", "AXP", "BA", "BAC", "BIIB", "BK", "BKNG",
    "BLK", "BMY", "BSX", "C", "CAT", "CHTR", "CL", "CMCSA", "COF", "COP",
    "COST", "CRM", "CSCO", "CVS", "CVX", "DHR", "DIS", "DOW", "DUK", "EMR",
    "EXC", "F", "FDX", "GD", "GE", "GILD", "GM", "GOOG", "GOOGL", "GS",
    "HON", "IBM", "INTC", "JNJ", "JPM", "KO", "LIN", "LLY", "LMT", "LOW",
    "MA", "MAR", "MCD", "MDLZ", "MDT", "MET", "MMM", "MO", "MRK", "MS",
    "MSFT", "NEE", "NFLX", "NKE", "NVDA", "ORCL", "PEP", "PFE", "PG", "PM",
    "PYPL", "QCOM", "RTX", "SBUX", "SCHW", "SO", "SPG", "T", "TGT", "TMO",
    "TMUS", "TSLA", "TXN", "UNH", "UNP", "UPS", "USB", "V", "VZ", "WBA",
    "WFC", "WMT", "XOM"
]

# **Functions:**

In [54]:
#takes a df and creates new columns that checks if the current 'High' price is higher than certain SMAs
def sma_filtering(df):
  filt20 = df["SMA20"] < df["High"]
  filt50 = df["SMA50"] < df["High"]
  filt100 = df["SMA100"] < df["High"]
  df["SMA20_Above"] = filt20
  df["SMA50_Above"] = filt50
  df["SMA100_Above"] = filt100
  df["above_avg"] = filt20 & filt50 & filt100
  return df

In [55]:
#takes as an input a stock name and period and returns the stock's df.
def stock_df(stock, period):
  dat = yf.Ticker(stock)
  df = dat.history(period=period).reset_index()
  df = df.loc[:, ['Date', 'Open', 'High', 'Low', 'Close']]
  return df

In [56]:
# takes a given stock and iterates over 'bunch-size" stocks at a time,
# applying 'func' to each bunch independantly
def iter(stock, bunch_size, func=None):
  i=0

  #taking the stock and converting it to a df.
  if type(stock) == str:
    cur_stock_df = stock_df(stock, '1y')
  elif type(stock) == pd.DataFrame:
    cur_stock_df = stock
  else:
    return 'not a valid dataFrame'

  #iterating while the bunch size equals the 'cur_bunch'
  #if they are not equal, it means we are on the last portion and we need to stop after this one.
  while True:
    cur_bunch = cur_stock_df.iloc[i:i+bunch_size, :]
    if len(cur_bunch) < bunch_size:
      break
    #if the user entered a function, apply it for each bunch.
    if func==bunch_mean:
      func(cur_stock_df, cur_bunch, bunch_size, i+bunch_size-1)
    elif func==weighted_bunch_mean:
      func(cur_stock_df, cur_bunch, bunch_size, i+bunch_size-1)
    i+=1
  return cur_stock_df

In [57]:
def bunch_mean(df, bunch, bunch_size, new_col_index):
  #creates a new column if not already exists.
  new_col = f"SMA{bunch_size}"
  if new_col not in df.columns:
        df[new_col] = None

  #calculates the mean of the current bunch.
  mean = bunch['High'].mean()

  #inputs the calculated mean in the corresponding column.
  df.loc[ new_col_index, f"SMA{bunch_size}"] = mean

In [58]:
def weighted_bunch_mean(df, bunch, bunch_size, new_col_index):
  weighted_mean=0
  index_sum=0
  new_col = f"WMA{bunch_size}"
  if new_col not in df.columns:
        df[new_col] = None

  #calculates the mean of the current bunch.
  for index, row in bunch.iterrows():
    index_sum += (index+1)
    weighted_mean += (index+1) * row['High']


  #inputs the calculated mean in the corresponding column.
  df.loc[ new_col_index, f"WMA{bunch_size}"] = (weighted_mean / index_sum)


In [59]:
def exponential_moving_average(df, n):
    # computes the exponential moving average (EMA) over n days
    alpha = 2 / (n + 1)
    ema_values = []       # list to store computed EMA values

    for i, price in enumerate(df["Close"]):
        if i == 0:
            ema_values.append(price)   # first EMA equals first price
        else:
            prev_ema = ema_values[-1]  # previous EMA
            ema_values.append(alpha * price + (1 - alpha) * prev_ema)  # EMA formula

    df[f"EMA_{n}"] = ema_values   # create new EMA column
    return df


In [60]:
# calculates WMA for a given window size using your weighted_bunch_mean function
def calc_wma_column(df, size):
    col = f"WMA{size}"
    df[col] = None

    for i in range(len(df)):
        # not enough rows to calculate WMA
        if i + 1 < size:
            continue

        # take the last 'size' rows
        bunch = df.iloc[i + 1 - size : i + 1]

        # use your function to place the result
        weighted_bunch_mean(df, bunch, size, i)

    return col

# Hull Moving Average
def hull_moving_average(df, n):
    half = int(n / 2)
    sqrt_n = int(np.sqrt(n))

    # calculate WMA(n)
    col_full = calc_wma_column(df, n)

    # calculate WMA(n/2)
    col_half = calc_wma_column(df, half)

    # create temporary values: 2*WMA(n/2) - WMA(n)
    df["HMA_temp"] = 2 * df[col_half] - df[col_full]

    # calculate WMA on the temporary values using sqrt(n)
    out_col = f"HMA{n}"
    df[out_col] = None

    for i in range(len(df)):
        # not enough rows for sqrt(n)
        if i + 1 < sqrt_n:
            continue

        # take the last sqrt(n) values
        bunch = df["HMA_temp"].iloc[i + 1 - sqrt_n : i + 1]

        # skip if there are missing values
        if bunch.isna().any():
            continue

        # compute WMA (same style as your function)
        weighted_mean = 0
        index_sum = 0

        for idx, val in enumerate(bunch):
            weight = idx + 1
            weighted_mean += weight * val
            index_sum += weight

        df.loc[i, out_col] = weighted_mean / index_sum

    return df


In [61]:
#finds the highest high price in the last n days for each row
def highest_high(df, window):
    df = df.copy()
    df[f'HH{window}'] = df['High'].rolling(window=window).max()
    return df

In [62]:
#finds the lowest low price in the last n days for each row
def lowest_low(df, window):
    df = df.copy()
    df[f'LL{window}'] = df['Low'].rolling(window=window).min()
    return df

In [63]:
def calculate_adx(df, n=14):

    df = df.copy()

    # Step 1: Price differences
    df['H_diff'] = df['High'].diff()
    df['L_diff'] = -df['Low'].diff()

    df['DM+'] = np.where((df['H_diff'] > df['L_diff']) & (df['H_diff'] > 0), df['H_diff'], 0)
    df['DM-'] = np.where((df['L_diff'] > df['H_diff']) & (df['L_diff'] > 0), df['L_diff'], 0)

    # Step 2: True Range
    df['TR1'] = df['High'] - df['Low']
    df['TR2'] = (df['High'] - df['Close'].shift()).abs()
    df['TR3'] = (df['Low'] - df['Close'].shift()).abs()
    df['TR'] = df[['TR1', 'TR2', 'TR3']].max(axis=1)

    # Step 3: Wilder smoothing
    df['TR_smooth'] = df['TR'].ewm(alpha=1/n, adjust=False).mean()
    df['DM+_smooth'] = df['DM+'].ewm(alpha=1/n, adjust=False).mean()
    df['DM-_smooth'] = df['DM-'].ewm(alpha=1/n, adjust=False).mean()

    # Step 4: DI+
    df['DI+'] = 100 * (df['DM+_smooth'] / df['TR_smooth'])
    df['DI-'] = 100 * (df['DM-_smooth'] / df['TR_smooth'])

    # Step 5: DX
    df['DX'] = 100 * ( (df['DI+'] - df['DI-']).abs() / (df['DI+'] + df['DI-']) )

    # Step 6: ADX
    df['ADX'] = df['DX'].ewm(alpha=1/n, adjust=False).mean()

    return df


# **Code:**

In [64]:
top10 = []
top10_dfs = []
i=0

for stock in sp100:
  if len(top10) == 10:
    break

  df = iter(stock, 20, bunch_mean)
  df = iter(df, 50, bunch_mean)
  df = iter(df, 100, bunch_mean)
  sma_filtering(df)

  df = iter(df, 20, weighted_bunch_mean)
  df = iter(df, 50, weighted_bunch_mean)
  df = iter(df, 100, weighted_bunch_mean)

  df = exponential_moving_average(df, 20)
  df = exponential_moving_average(df, 50)

  df = hull_moving_average(df, 20)
  df = hull_moving_average(df, 50)

  df = highest_high(df,20)
  df = lowest_low(df,20)

  df = calculate_adx(df, n=14)

  if(df.iloc[-1,-1]):
    top10.append(stock)
    top10_dfs.append(df)
  i+=1

top10

['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'AMD', 'AMGN', 'AMT', 'AMZN']

In [65]:
top10_dfs[0]

,Date,Open,High,Low,Close,SMA20,SMA50,SMA100,SMA20_Above,SMA50_Above,...,TR2,TR3,TR,TR_smooth,DM+_smooth,DM-_smooth,DI+,DI-,DX,ADX
0,2024-11-20 00:00:00-05:00,227.033178,228.894754,224.872950,227.968948,None,None,None,False,False,...,NaN,NaN,4.021804,4.021804,0.000000,0.000000,0.000000,0.000000,NaN,NaN
1,2024-11-21 00:00:00-05:00,227.849484,229.123719,224.693758,227.491104,None,None,None,False,False,...,1.154771,3.275190,4.429961,4.050958,0.016355,0.000000,0.403724,0.000000,100.000000,100.000000
2,2024-11-22 00:00:00-05:00,227.033174,229.681201,227.033174,228.835022,None,None,None,False,False,...,2.190097,0.457930,2.648027,3.950748,0.055007,0.000000,1.392308,0.000000,100.000000,100.000000
3,2024-11-25 00:00:00-05:00,230.417878,232.199812,228.705621,231.821518,None,None,None,False,False,...,3.364790,0.129401,3.494191,3.918137,0.230978,0.000000,5.895106,0.000000,100.000000,100.000000
4,2024-11-26 00:00:00-05:00,232.279457,234.509377,232.279457,234.001663,None,None,None,False,False,...,2.687859,0.457939,2.687859,3.830260,0.379449,0.000000,9.906607,0.000000,100.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,2025-11-14 00:00:00-05:00,271.049988,275.959991,269.600006,272.410004,270.649927,258.056559,239.06462,True,True,...,3.009979,3.350006,6.359985,5.249077,1.359932,0.744245,25.908017,14.178587,29.260225,32.157218
247,2025-11-17 00:00:00-05:00,268.820007,270.489990,265.730011,267.459991,270.968228,258.66801,239.747377,False,True,...,1.920013,6.679993,6.679993,5.351285,1.262794,0.967513,23.597953,18.080008,13.239479,30.805951
248,2025-11-18 00:00:00-05:00,269.989990,270.709991,265.320007,267.440002,271.252073,259.311235,240.426547,False,True,...,3.250000,2.139984,5.389984,5.354049,1.172594,0.927691,21.901072,17.326899,11.660489,29.438418
249,2025-11-19 00:00:00-05:00,265.529999,272.209991,265.500000,268.559998,271.732801,260.111537,241.079104,True,True,...,4.769989,1.940002,6.709991,5.450902,1.195980,0.861427,21.940960,15.803385,16.260913,28.497167


In [66]:
#check if the code works correctly
df_copy = df.copy()
df_copy = df_copy.head(40)
df_copy = highest_high(df_copy, 20)
df_copy = lowest_low(df_copy, 20)
print(df_copy)


                        Date        Open        High         Low       Close  \
0  2024-11-20 00:00:00-05:00  202.979996  203.130005  199.449997  202.880005   
1  2024-11-21 00:00:00-05:00  203.490005  203.490005  195.750000  198.380005   
2  2024-11-22 00:00:00-05:00  198.250000  199.259995  196.750000  197.119995   
3  2024-11-25 00:00:00-05:00  199.279999  201.949997  199.000000  201.449997   
4  2024-11-26 00:00:00-05:00  201.899994  208.000000  201.789993  207.860001   
5  2024-11-27 00:00:00-05:00  206.979996  207.639999  205.050003  205.740005   
6  2024-11-29 00:00:00-05:00  205.830002  208.199997  204.589996  207.889999   
7  2024-12-02 00:00:00-05:00  209.960007  212.990005  209.509995  210.710007   
8  2024-12-03 00:00:00-05:00  210.309998  214.020004  209.649994  213.440002   
9  2024-12-04 00:00:00-05:00  215.960007  220.000000  215.750000  218.160004   
10 2024-12-05 00:00:00-05:00  218.029999  222.149994  217.300003  220.550003   
11 2024-12-06 00:00:00-05:00  220.750000

In [67]:
i = 0
for df in top10_dfs:
    df['Date'] = pd.to_datetime(df['Date'])

    fig = go.Figure(data=go.Scatter(x=df['Date'], y=df['High'], mode='lines', name='Price'))
    fig.add_scatter(x=df['Date'], y=df['SMA20'], mode='lines', name='SMA 20')
    fig.add_scatter(x=df['Date'], y=df['SMA50'], mode='lines', name='SMA 50')
    fig.add_scatter(x=df['Date'], y=df['SMA100'], mode='lines', name='SMA 100')


    high_above_sma = df['High'].where(df['above_avg'], other=np.nan)

    fig.add_scatter(
        x=df['Date'],
        y=high_above_sma,
        mode='lines',
        line=dict(color='white', width=3, dash='dot'),
        name='Above all SMAs'
    )

    fig.update_layout(
        title=f"{top10[i]} Stock",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_dark"
    )
    fig.update_xaxes(tickformat="%m-%Y")
    fig.show()
    i += 1


In [68]:
i = 0
for df in top10_dfs:

    df['Date'] = pd.to_datetime(df['Date'])

    fig = go.Figure(data=go.Scatter(x=df['Date'], y=df['High'], mode='lines', name='Price', line=dict(color='white')))

    fig.add_scatter(x=df['Date'], y=df['SMA20'], mode='lines', name='SMA 20', line=dict(color='green'))
    fig.add_scatter(x=df['Date'], y=df['WMA20'], mode='lines', name='WMA 20', line=dict(color='blue'))
    fig.add_scatter(x=df['Date'], y=df['EMA_20'], mode='lines', name='EMA 20', line=dict(color='red'))
    fig.add_scatter(x=df['Date'], y=df['HMA20'], mode='lines', name='HMA 20', line=dict(color='cyan'))

    fig.update_layout(
        title=f"{top10[i]} Stock",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_dark"
    )

    fig.update_xaxes(tickformat="%m-%Y")
    fig.show()

    i += 1


In [69]:
i = 0
for df in top10_dfs:

    df['Date'] = pd.to_datetime(df['Date'])

    fig = go.Figure(data=go.Scatter(x=df['Date'], y=df['High'], mode='lines', name='Price', line=dict(color='white')))

    fig.add_scatter(x=df['Date'], y=df['SMA50'], mode='lines', name='SMA 50', line=dict(color='green'))
    fig.add_scatter(x=df['Date'], y=df['WMA50'], mode='lines', name='WMA 50', line=dict(color='blue'))
    fig.add_scatter(x=df['Date'], y=df['EMA_50'], mode='lines', name='EMA 50', line=dict(color='red'))
    fig.add_scatter(x=df['Date'], y=df['HMA50'], mode='lines', name='HMA 50', line=dict(color='cyan'))

    fig.update_layout(
        title=f"{top10[i]} Stock",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_dark"
    )

    fig.update_xaxes(tickformat="%m-%Y")
    fig.show()

    i += 1


In [70]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

i = 0
for df in top10_dfs:

    df['Date'] = pd.to_datetime(df['Date'])
    df = df.replace({None: np.nan})  # make comparisons safe

    # price above ALL moving averages
    above_20 = (df['High'] > df['SMA20']) & (df['High'] > df['WMA20']) & (df['High'] > df['EMA_20']) & (df['High'] > df['HMA20'])
    high_above_20 = df['High'].where(above_20, other=np.nan)

    above_50 = (df['High'] > df['SMA50']) & (df['High'] > df['WMA50']) & (df['High'] > df['EMA_50']) & (df['High'] > df['HMA50'])
    high_above_50 = df['High'].where(above_50, other=np.nan)

    fig = make_subplots(rows=1, cols=2, subplot_titles=("Window 20", "Window 50"))

    # ================= Window 20 =================
    fig.add_trace(go.Scatter(x=df['Date'], y=df['High'], mode='lines', name='Price', line=dict(color='white')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA20'], mode='lines', name='SMA 20', line=dict(color='green')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['WMA20'], mode='lines', name='WMA 20', line=dict(color='blue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['EMA_20'], mode='lines', name='EMA 20', line=dict(color='red')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['HMA20'], mode='lines', name='HMA 20', line=dict(color='cyan')), row=1, col=1)

    # dashed white line
    fig.add_scatter(x=df['Date'], y=high_above_20, mode='lines',
                    line=dict(color='orange', width=3, dash='dot'),
                    name='Above all MAs 20', row=1, col=1)

    # ================= Window 50 =================
    fig.add_trace(go.Scatter(x=df['Date'], y=df['High'], mode='lines', name='Price', line=dict(color='white')), row=1, col=2)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA50'], mode='lines', name='SMA 50', line=dict(color='green')), row=1, col=2)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['WMA50'], mode='lines', name='WMA 50', line=dict(color='blue')), row=1, col=2)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['EMA_50'], mode='lines', name='EMA 50', line=dict(color='red')), row=1, col=2)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['HMA50'], mode='lines', name='HMA 50', line=dict(color='cyan')), row=1, col=2)

    # dashed white line
    fig.add_scatter(x=df['Date'], y=high_above_50, mode='lines',
                    line=dict(color='orange', width=3, dash='dot'),
                    name='Above all MAs 50', row=1, col=2)

    fig.update_layout(title=f"{top10[i]} — Comparison: Window 20 vs Window 50", template="plotly_dark")
    fig.update_xaxes(tickformat="%m-%Y")

    fig.show()
    i += 1


/tmp/ipython-input-911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [72]:
i = 0
for df in top10_dfs:

    df['Date'] = pd.to_datetime(df['Date'])

    fig = go.Figure()

    fig.add_scatter(
        x=df['Date'],
        y=df['ADX'],
        mode='lines',
        name='ADX 14',
        line=dict(color='red')
    )

    fig.update_layout(
        title=f"{top10[i]} — ADX Indicator",
        xaxis_title="Date",
        yaxis_title="ADX Value",
        template="plotly_dark"
    )

    fig.update_xaxes(tickformat="%m-%Y")
    fig.show()

    i += 1
